# Welcome to Full Stack Machine Learning's Week 4 Project!

In the final week, you will return to the workflow you built last week on the [taxi dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page). 

## Task 1: Deploy the champion
Use what you have learned in the last two weeks to make necessary modifications and to deploy your latest version of the `TaxiFarePrediction` flow to Argo. Use `--branch champion` to denote this deployment as the champion model.

In [ ]:
! install metaflow==2.8.6  # there are issues with latest 2.9 version

In [6]:
%%writefile ../flows/cloud/taxiprediction_champion.py
from metaflow import FlowSpec, step, S3, card, conda_base, current, Parameter, Flow, project, trigger, retry, timeout,catch
from metaflow.cards import Markdown, Table, Image, Artifact

# URL = 'https://outerbounds-datasets.s3.us-west-2.amazonaws.com/taxi/latest.parquet'
URL = 's3://outerbounds-datasets/taxi/latest.parquet'
DATETIME_FORMAT = '%Y-%m-%d %H:%M:%S'

@trigger(events=['s3'])
@conda_base(libraries={'pandas': '1.4.2', 'pyarrow': '11.0.0', 'numpy': '1.21.2', 'scikit-learn': '1.1.2'})
@project(name="taxi_fare_prediction")
class TaxiFarePredictionChampion(FlowSpec):

    data_url = Parameter("data_url", default=URL)

    def transform_features(self, df):

        obviously_bad_data_filters = [
            df.fare_amount > 0,         # fare_amount in US Dollars
            df.trip_distance <= 100,    # trip_distance in miles
            df.trip_distance > 0,
            df.passenger_count >0,
            df.fare_amount >0
            # TODO: add some logic to filter out what you decide is bad data!
            # TIP: Don't spend too much time on this step for this project though, it practice it is a never-ending process.

        ]
        for f in obviously_bad_data_filters:
            df = df[f]        
        df= df.dropna()
        return df
    
    @catch(var="read_failure")
    @retry(times=4)
    @timeout(minutes=10)
    @step
    def start(self):

        import pandas as pd
        from sklearn.model_selection import train_test_split

        self.df = self.transform_features(pd.read_parquet(self.data_url))

        # NOTE: we are split into training and validation set in the validation step which uses cross_val_score.
        # This is a simple/naive way to do this, and is meant to keep this example simple, to focus learning on deploying Metaflow flows.
        # In practice, you want split time series data in more sophisticated ways and run backtests. 
        self.X = self.df["trip_distance"].values.reshape(-1, 1)
        self.y = self.df["total_amount"].values
        self.next(self.linear_model)

    @step
    def linear_model(self):
        "Fit a single variable, linear model to the data."
        from sklearn.linear_model import LinearRegression

        self.model = LinearRegression()

        self.next(self.validate)

    def gather_sibling_flow_run_results(self):

        # storage to populate and feed to a Table in a Metaflow card
        rows = []

        # loop through runs of this flow 
        for run in Flow(self.__class__.__name__):
            if run.id != current.run_id:
                if run.successful:
                    icon = "✅" 
                    msg = "OK"
                    score = str(run.data.scores.mean())
                else:
                    icon = "❌"
                    msg = "Error"
                    score = "NA"
                    for step in run:
                        for task in step:
                            if not task.successful:
                                msg = task.stderr
                row = [Markdown(icon), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(score), Markdown(msg)]
                rows.append(row)
            else:
                rows.append([Markdown("✅"), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(str(self.scores.mean())), Markdown("This run...")])
        return rows
                
    
    @card(type="corise")
    @step
    def validate(self):
        from sklearn.model_selection import cross_val_score
        self.scores = cross_val_score(self.model, self.X, self.y, cv=5)
        current.card.append(Markdown("# Taxi Fare Prediction Results"))
        current.card.append(Table(self.gather_sibling_flow_run_results(), headers=["Pass/fail", "Run ID", "Created At", "R^2 score", "Stderr"]))
        self.next(self.end)

    @step
    def end(self):
        self.model_type = "baseline"
        print("Success!")


if __name__ == "__main__":
    TaxiFarePredictionChampion()

Overwriting ../flows/cloud/taxiprediction_champion.py


In [7]:
! python ../flows/cloud/taxiprediction_champion.py --environment=conda --production --branch champion argo-workflows create 

Metaflow 2.9.1+ob(v1) executing TaxiFarePredictionChampion for user:sandbox
Project: taxi_fare_prediction, Branch: prod.champion
2023-05-21 15:15:09.461 Creating local datastore in current directory (/home/workspace/full-stack-ml-metaflow-corise-week-4/project/.metaflow)
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying taxifareprediction.prod.champion.taxifarepredictionchampion to Argo Workflows...
It seems this is the first time you are deploying taxifareprediction.prod.champion.taxifarepredictionchampion to Argo Workflows.

A new production token generated.

The namespace of this production flow is
    production:mfprj-ovzw7jjg7psagpyw-0-pcke
To analyze results of this production flow add this line in your notebooks:
    namespace("production:mfprj-ovzw7jjg7psagpyw-0-pcke")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize mfprj-ovzw7jjg

In [13]:
! python ../flows/cloud/taxiprediction_champion.py --environment=conda --production --branch champion argo-workflows trigger

Metaflow 2.8.6+ob(v1) executing TaxiFarePredictionChampion for user:sandbox
Project: taxi_fare_prediction, Branch: prod.champion
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Workflow taxifareprediction.prod.champion.taxifarepredictionchampion triggered on Argo Workflows (run-id argo-taxifareprediction.prod.champion.taxifarepredictionchampiox6sdl).
See the run in the UI at https://ui-pw-690062252.outerbounds.dev/TaxiFarePredictionChampion/argo-taxifareprediction.prod.champion.taxifarepredictionchampiox6sdl


## Task 2: Build the challenger
Develop a second model, by using the same `TaxiFarePrediction` architecture. Then, deploy the flow to Argo as the `--branch challenger`. 
<br>
<br>
Hint: Modify the `linear_model` step. 
<br>
Bonus: Write a paragraph summary of how you developed the second model and tested it before deploying the challenger flow. Let us know in Slack what you found challenging about the task? 

In [9]:
%%writefile ../flows/cloud/taxiprediction_challenger.py
from metaflow import FlowSpec, step, S3, card, conda_base, current, Parameter, Flow, project, trigger, retry, timeout,catch
from metaflow.cards import Markdown, Table, Image, Artifact

URL = 'https://outerbounds-datasets.s3.us-west-2.amazonaws.com/taxi/latest.parquet'
#URL = 's3://outerbounds-datasets/taxi/latest.parquet'
DATETIME_FORMAT = '%Y-%m-%d %H:%M:%S'

@trigger(events=['s3'])
@conda_base(libraries={'pandas': '1.4.2', 'pyarrow': '11.0.0', 'numpy': '1.21.2', 'scikit-learn': '1.1.2'})
@project(name="taxi_fare_prediction")
class TaxiFarePredictionChallenger(FlowSpec):

    data_url = Parameter("data_url", default=URL)

    def transform_features(self, df):

        obviously_bad_data_filters = [
            df.fare_amount > 0,         # fare_amount in US Dollars
            df.trip_distance <= 100,    # trip_distance in miles
            df.trip_distance > 0,
            df.passenger_count >0,
            df.fare_amount >0
            # TODO: add some logic to filter out what you decide is bad data!
            # TIP: Don't spend too much time on this step for this project though, it practice it is a never-ending process.

        ]
        for f in obviously_bad_data_filters:
            df = df[f]        
        df= df.dropna()
        return df
    
    @catch(var="read_failure")
    @retry(times=4)
    @timeout(minutes=10)
    @step
    def start(self):

        import pandas as pd
        from sklearn.model_selection import train_test_split

        self.df = self.transform_features(pd.read_parquet(self.data_url))

        # NOTE: we are split into training and validation set in the validation step which uses cross_val_score.
        # This is a simple/naive way to do this, and is meant to keep this example simple, to focus learning on deploying Metaflow flows.
        # In practice, you want split time series data in more sophisticated ways and run backtests. 
        self.X = self.df["trip_distance"].values.reshape(-1, 1)
        self.y = self.df["total_amount"].values
        self.next(self.linear_model)

    @step
    def linear_model(self):

        # from sklearn.linear_model import LinearRegression
        #self.model = LinearRegression()

        from sklearn.svm import LinearSVC  # support vector
        self.model = LinearSVC(dual=False)

        self.next(self.validate)

    def gather_sibling_flow_run_results(self):

        # storage to populate and feed to a Table in a Metaflow card
        rows = []

        # loop through runs of this flow 
        for run in Flow(self.__class__.__name__):
            if run.id != current.run_id:
                if run.successful:
                    icon = "✅" 
                    msg = "OK"
                    score = str(run.data.scores.mean())
                else:
                    icon = "❌"
                    msg = "Error"
                    score = "NA"
                    for step in run:
                        for task in step:
                            if not task.successful:
                                msg = task.stderr
                row = [Markdown(icon), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(score), Markdown(msg)]
                rows.append(row)
            else:
                rows.append([Markdown("✅"), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(str(self.scores.mean())), Markdown("This run...")])
        return rows
                
    
    @card(type="corise")
    @step
    def validate(self):
        from sklearn.model_selection import cross_val_score
        self.scores = cross_val_score(self.model, self.X, self.y, cv=5)
        current.card.append(Markdown("# Taxi Fare Prediction Results"))
        current.card.append(Table(self.gather_sibling_flow_run_results(), headers=["Pass/fail", "Run ID", "Created At", "R^2 score", "Stderr"]))
        self.next(self.end)

    @step
    def end(self):
        self.model_type = "baseline"
        print("Success!")


if __name__ == "__main__":
    TaxiFarePredictionChallenger()

Overwriting ../flows/cloud/taxiprediction_challenger.py


In [10]:
! python ../flows/cloud/taxiprediction_challenger.py --environment=conda --production --branch challenger argo-workflows create

Metaflow 2.9.1+ob(v1) executing TaxiFarePredictionChallenger for user:sandbox
Project: taxi_fare_prediction, Branch: prod.challenger
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying taxifareprediction.prod.challenger.taxifarepredictionchallenger to Argo Workflows...
It seems this is the first time you are deploying taxifareprediction.prod.challenger.taxifarepredictionchallenger to Argo Workflows.

A new production token generated.

The namespace of this production flow is
    production:mfprj-6iffsxtybx6fkjku-0-crtx
To analyze results of this production flow add this line in your notebooks:
    namespace("production:mfprj-6iffsxtybx6fkjku-0-crtx")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize mfprj-6iffsxtybx6fkjku-0-crtx
when deploying this flow to Argo Workflows for the first time.
See "Organizing Results" at https://docs.metaflow.or

In [14]:
! python ../flows/cloud/taxiprediction_challenger.py --environment=conda --production --branch challenger argo-workflows trigger

Metaflow 2.8.6+ob(v1) executing TaxiFarePredictionChallenger for user:sandbox
Project: taxi_fare_prediction, Branch: prod.challenger
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Workflow taxifareprediction.prod.challenger.taxifarepredictionchallenger triggered on Argo Workflows (run-id argo-taxifareprediction.prod.challenger.taxifarepredictionchallfz9qq).
See the run in the UI at https://ui-pw-690062252.outerbounds.dev/TaxiFarePredictionChallenger/argo-taxifareprediction.prod.challenger.taxifarepredictionchallfz9qq


## Task 3: Analyze the results
Return to this notebook, and read in the results of the challenger and champion flow using the Metaflow Client API.
<br><br>

#### Questions
- Does your model perform better on the metrics you selected? 
- Think about your day job, how would you go about assessing whether to roll forward the production "champion" to your new model? 
    - What gives you confidence one model is better than another?
    - What kinds of information do you need to monitor to get buy-in from stakeholders that model A is preferable to model B?  

In [15]:
from metaflow import Flow, namespace
import numpy as np

CHAMPION_MODEL_NAMESPACE = 'production:mfprj-ovzw7jjg7psagpyw-0-pcke'
CHALLENGER_MODEL_NAMESPACE = 'production:mfprj-6iffsxtybx6fkjku-0-crtx'

best_score = -1; winner = None; winner_namespace = None
for model, n in zip(('Champion','Challenger'),[CHAMPION_MODEL_NAMESPACE, CHALLENGER_MODEL_NAMESPACE]):
    namespace(n)
    run = Flow('TaxiFarePrediction'+model).latest_successful_run
    acc_score = np.mean(run.data.scores)
    print("Latest {} model had accuracy = {}%".format(run.data.model_type, acc_score))
    if acc_score > best_score:
        best_score = acc_score
        winner = run.data.model_type
        winner_namespace = n
print(f"Winner is the {winner} model with accuracy of {best_score:.2%}%")


AttributeError: 'NoneType' object has no attribute 'data'

Both methods of accessing the parquet file fail, so I have enough of this.  
> FileNotFoundError: [Errno 2] No such file or directory: '"s3://outerbounds-datasets/taxi/latest.parquet"' \
FileNotFoundError: [Errno 2] No such file or directory: '"https://outerbounds-datasets.s3.us-west-2.amazonaws.com/taxi/latest.parquet"'

## CONGRATULATIONS! 🎉✨🍾
If you made it this far, you have completed the Full Stack Machine Learning Corise course. 
We are so glad that you chose to learn with us, and hope to see you again in future courses. Stay tuned for more content and come join us in [Slack](http://slack.outerbounds.co/) to keep learning about Metaflow!